In [2]:
from datasets import load_dataset
import tiktoken

c:\Users\steve\repos\BundtCake\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = load_dataset("Stegvean/CakeGenomes")
dataset

# split dataset
split_dataset = dataset["train"].train_test_split(test_size=0.1, shuffle=True)
split_dataset["val"] = split_dataset.pop('test') 
[split_dataset["train"][i] for i in range(10)]